In [49]:
!pip install llama_cloud_services openpyxl

In [50]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from financial_schemas_end_v8 import EndowmentAndInvestmentLevels_2024_25  #This could be adjusted through schemas.py
#from dotenv import load_dotenv


In [51]:
PDF_ROOT = "private_universities/university_pdfs"
OUTPUT_ROOT = "output_fin_nk_v8"
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "56843d2c-7e9b-445d-b634-9833dd1cb4db" #Different based on your LLamaCloud account
api_key = os.getenv("LLAMACLOUD_API_KEY")

In [ ]:
extractor = LlamaExtract(
    api_key="llx-63CU3PdyDo0d230ureocmy9JOHgnPwYgE2HETi55DqzYCIpy",  
    project_id="8c10e62e-3810-4193-915d-d2d11105826d"
)



#agent = extractor.create_agent(name = "endowment-parser-2024", data_schema=EndowmentAndInvestmentLevels_2024_25)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
agent.data_schema = EndowmentAndInvestmentLevels_2024_25.schema()
agent.save()
agent = extractor.get_agent(id = AGENT_ID)


/var/folders/m1/8j3nd3m95y1brb0j52fx43fr0000gn/T/ipykernel_2739/1285879971.py:13: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  agent.data_schema = EndowmentAndInvestmentLevels_2024_25.schema()


Extracting files: 100%|██████████| 1/1 [00:06<00:00,  6.86s/it]


In [53]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'endowment_net_assets_eoy_total': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Total endowment net assets for the 2024–2025 fiscal year (in thousands). Only extract from a table labeled 'Changes in Endowment Net Assets' in the Notes section. **Include only values clearly labeled as 2024, FY2024, or 'as of June 30, 2024'. Ignore 2023 and earlier values.** Do not extract from general balance sheets or summaries. Values must be standardized to $000s."},
  'endowment_net_assets_eoy_with_donor_restrictions': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Total donor-restricted endowment net assets as of the end of fiscal year 2024–2025 (in thousands). Must be from a 'Changes in Endowment Net Assets' table in the Notes section. **Exclude any data from 2023 or earlier.**"},
  'endowment_net_assets_eoy_with_donor_restrictions_temporarily_restricted': {'anyOf': [{'type': 'integer'},
    {'typ

The following two cell blocks extract all schools' info into one excel file per school.

In [54]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [55]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

Processing school: BRADLEY_UNIVERSITY
Extracting data from Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf
Extracting data from Audited_Financial_Statements_or_ACFR__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__541_KB_.pdf
Saved output to output_fin_nk_v8/BRADLEY_UNIVERSITY.xlsx
Processing school: CORNELL_UNIVERSITY
Extracting data from 2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__788_KB_.pdf
Extracting data from 2024_Operating_Data_for_the_year_ended_06_30_2024__109_KB_.pdf
Extracting data from Incorporate_OS_by_Reference_as_of_04_25_2024__2.4_MB_.pdf
Saved output to output_fin_nk_v8/CORNELL_UNIVERSITY.xlsx
Processing school: CULINARY_INSTITUTE_OF_AMERICA_THE
Extracting data from 2024_Annual_Report_-_Corrected_for_the_year_ended_05_31_2024__130_KB_.pdf
Extracting data from 2024_Annual_Report_for_the_year_ended_05_31_2024__129_KB_.pdf
Extracting data from 2024_Audited_Financial_Statements_fo

Extracting files:   0%|          | 0/1 [05:04<?, ?it/s]


Skipped 2024_Operating_Data_for_the_year_ended_06_30_2024__244_KB_.pdf: Request timed out: 
Saved output to output_fin_nk_v8/NEW_YORK_UNIVERSITY.xlsx
Processing school: PRESIDENT___FELLOWS_OF_HARVARD_COLLEGE
Extracting data from Harvard_University_Audited_Financial_Information_for_the_year_ended_06_30_2024__10.6_MB_.pdf
Extracting data from Harvard_University_Financial_Report_for_the_year_ended_06_30_2024__10.6_MB_.pdf
Extracting data from Harvard_University_Student_Applications_and_Enrollment_for_the_year_ended_06_30_2024__557_KB_.pdf
Saved output to output_fin_nk_v8/PRESIDENT___FELLOWS_OF_HARVARD_COLLEGE.xlsx
Processing school: STEVENSON_UNIVERSITY_INC
Extracting data from 2024_Annual_Compliance_Certificate__EagleBank__for_the_year_ended_06_30_2024__3_MB_.pdf
Extracting data from 2024_Annual_Compliance_Certificate_for_the_year_ended_06_30_2024__2.8_MB_.pdf
Extracting data from 2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__348_KB_.pdf
Extracting data from 2024_Opera

Extracting files:   0%|          | 0/1 [09:39<?, ?it/s]


Saved output to output_fin_nk_v8/STEVENSON_UNIVERSITY_INC.xlsx
Processing school: STEVENS_INSTITUTE_OF_TECHNOLOGY
Extracting data from Annual_Report_for_the_year_ended_06_30_2024__216_KB_.pdf
Extracting data from Audit_Financial_Statement_for_the_year_ended_06_30_2024__626_KB_.pdf
Saved output to output_fin_nk_v8/STEVENS_INSTITUTE_OF_TECHNOLOGY.xlsx
Processing school: ST_LOUIS_UNIVERSITY_US
Extracting data from Amendment_to_Continuing_Disclosure_Undertaking_dated_01_05_2024__392_KB_.pdf
Extracting data from Audited_Financials_and_Operating_Data_for_the_year_ended_06_30_2024_Document1__561_KB_.pdf
Extracting data from Audited_Financials_and_Operating_Data_for_the_year_ended_06_30_2024_Document2__174_KB_.pdf
Saved output to output_fin_nk_v8/ST_LOUIS_UNIVERSITY_US.xlsx
Extraction complete.


The following cell block extracts all the schools' info into one excel sheet but in different tabs.

In [56]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")

Extracting data from BRADLEY_UNIVERSITY/Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf
Extracting data from BRADLEY_UNIVERSITY/Audited_Financial_Statements_or_ACFR__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__541_KB_.pdf
Extracting data from CORNELL_UNIVERSITY/2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__788_KB_.pdf
Extracting data from CORNELL_UNIVERSITY/2024_Operating_Data_for_the_year_ended_06_30_2024__109_KB_.pdf
Extracting data from CORNELL_UNIVERSITY/Incorporate_OS_by_Reference_as_of_04_25_2024__2.4_MB_.pdf
Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Annual_Report_-_Corrected_for_the_year_ended_05_31_2024__130_KB_.pdf
Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Annual_Report_for_the_year_ended_05_31_2024__129_KB_.pdf
Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Audited_Financial_Statements_for_the_year_ended_05_31_2024__277_KB_.pdf
Extra

In [57]:
#Combine all the tabs into one sheet if wanted
file_path   = "output_fin_nk_v8/all_schools.xlsx"
output_path = "output_fin_nk_v8/all_schools_combined.xlsx"

raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

school_series = {
    school: df.iloc[:, 0]                      # first (only) value column
    for school, df in raw.items()
}

df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"                 
df_comb.insert(0, "Year", "2023‑2024")

# df_comb.loc['California_state_university', 'Undergraduate_Headcount'] = None 
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

print("Saved:", output_path)

Saved: output_fin_nk_v8/all_schools_combined.xlsx
